## Collect All Article Data

In [6]:
# Get the articles
import requests
import json

url = "https://api-hoaxy.p.rapidapi.com/articles"

headers = {
    'x-rapidapi-host': "api-hoaxy.p.rapidapi.com",
    'x-rapidapi-key': "6dda23c3b6msh40fb43640818777p173117jsn2e799b5b84a7"
    }

vbm_query = "(mailin or ballot or votebymail)"

def fetch_articles(querystring):
    query = {"sort_by":"relevant","use_lucene_syntax":"true","query":querystring}
    response = requests.request("GET", url, headers=headers, params=query)
    return json.loads(response.text)

# print(fetch_articles(vbm_query)['articles'])

In [5]:
from datetime import date
from dateutil.relativedelta import relativedelta

def collect_artcicles_over_time(base_query, start_yr=2016, end_yr=2020, step=7):
    articles = []
    start = date(start_yr, 1, 1)
    while start.year < end_yr + 1:
        end = start + relativedelta(days=step)
        # Query for mail in voting articles published within this time period.
        querystring = base_query + f" AND date_published:[{start.isoformat()} TO {end.isoformat()}]"
        res = fetch_articles(querystring)
        if 'articles' in res.keys():
            articles.extend(res['articles'])
        # Update the start date so there's no overlap.
        start = end + relativedelta(days=+1)
    return articles

# articles_18 = collect_artcicles_over_time(vbm_query, start_yr=2018, end_yr=2018)
articles_20 = collect_artcicles_over_time(vbm_query, start_yr=2020, end_yr=2020)

KeyboardInterrupt: 

In [ ]:
# raw_articles = articles_18 + articles_20
raw_articles = articles_20
print("num articles", len(raw_articles))
ids = set(map(lambda x: x["id"], raw_articles))
print("num unique ids", len(set(ids)))

In [12]:
from datetime import date
from dateutil.relativedelta import relativedelta
articles = []
# Query for mail in voting articles published within this time period.
querystring = vbm_query + f" AND date_published:[{date(2020, 11, 6).isoformat()} TO {date(2020,11,8).isoformat()}]"
res = fetch_articles(querystring)
if 'articles' in res.keys():
    articles.extend(res['articles'])
raw_articles = articles

## Save Article Data

In [13]:
from datetime import datetime
import pytz
import pandas as pd

midway_date = pytz.utc.localize(datetime(2020, 7, 2))

class Article:
    def __init__(self, article_dict):
        self.id = article_dict['id']
        self.title = article_dict['title'].replace(",", "")
        self.date_published = article_dict['date_published']
        self.url = article_dict['canonical_url'].split(",")[0]
        self.domain = article_dict['domain']
        self.num_tweets = article_dict['number_of_tweets']
        self.score = article_dict['score']
        self.site_type = article_dict['site_type']
        self.which_half = 1 if pd.to_datetime(self.date_published) < midway_date else 2
        
    
    @classmethod
    def write_csv_col_names(cls, csv):
        csv.write(f"id,title,date_published,which_half,url,domain,num_tweets,score,site_type\n")
    
    def write_to_csv(self, csv):
        csv.write(f"{self.id},{self.title},{self.date_published},{self.which_half},{self.url},{self.domain},{self.num_tweets},{self.score},{self.site_type}\n")
    
    def __str__(self):
        return f"Article({self.id}, {self.title}, {self.url})"
    
articles = [Article(a) for a in raw_articles]

In [14]:
article_data_path = "../data/2020_11_6_articles.csv"

with open(article_data_path, "w", encoding='utf-8') as f:
    Article.write_csv_col_names(f)
    for a in articles:
        a.write_to_csv(f)

In [15]:
import pandas as pd
article_df = pd.read_csv(article_data_path).drop_duplicates(["id"], ignore_index=True)
print(article_df.shape)

(100, 9)


##### 